In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

In [4]:
# Các mô hình
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb

import joblib # Để lưu mô hình

/home/nhat/projectcuoiky/.venv/lib/python3.13/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [5]:
# --- 1. Tải và Chuẩn bị Dữ liệu ---
df = pd.read_csv('/home/nhat/projectcuoiky/data/pdf_features.csv') # Cập nhật đường dẫn nếu cần

# Kiểm tra các cột có thể không phù hợp
print("Thông tin ban đầu của DataFrame:")
df.info()
print("\n5 dòng đầu tiên:")
print(df.head())

Thông tin ban đầu của DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11101 entries, 0 to 11100
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Page           11101 non-null  int64  
 1   Encrypt        11101 non-null  int64  
 2   ObjStm         11101 non-null  int64  
 3   JS             11101 non-null  int64  
 4   JavaScript     11101 non-null  int64  
 5   AA             11101 non-null  int64  
 6   OpenAction     11101 non-null  int64  
 7   AcroForm       11101 non-null  int64  
 8   JBIG2Decode    11101 non-null  int64  
 9   RichMedia      11101 non-null  int64  
 10  Launch         11101 non-null  int64  
 11  EmbeddedFile   11101 non-null  int64  
 12  XFA            11101 non-null  int64  
 13  Colors_gt_224  11101 non-null  int64  
 14  obj            11101 non-null  int64  
 15  endobj         11101 non-null  int64  
 16  stream         11101 non-null  int64  
 17  endstream      11

In [6]:
le = LabelEncoder()
df['label_numeric'] = le.fit_transform(df['label'])
# In ra để xem ánh xạ (ví dụ: 'benign' -> 0, 'malicious' -> 1)
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(f"\nÁnh xạ nhãn: {label_mapping}")
print(f"Các lớp đã được mã hóa: {le.classes_}") # Hiển thị các lớp gốc


Ánh xạ nhãn: {'benign': np.int64(0), 'malicious': np.int64(1)}
Các lớp đã được mã hóa: ['benign' 'malicious']


In [8]:
print(df.head(5))

   Page  Encrypt  ObjStm  JS  JavaScript  AA  OpenAction  AcroForm  \
0     1        0       0   0           0   0           0         0   
1     1        0       0   0           0   0           0         0   
2     4        0       6   0           0   0           0         0   
3     1        0       0   0           0   0           0         1   
4     6        0      25   0           0   0           0         2   

   JBIG2Decode  RichMedia  ...  stream  endstream  xref  trailer  startxref  \
0            0          0  ...       3          3     2        2          2   
1            0          0  ...       2          2     1        1          1   
2            0          0  ...      41         41     0        0          3   
3            0          0  ...      17         17     2        2          2   
4            0          0  ...     146        146     0        0          4   

                                            filepath          filename  \
0  /home/remnux/Desktop/extrac

In [9]:
# Loại bỏ các cột không cần thiết và các cột không thông tin
features_to_drop = ['filepath', 'filename', 'label', '_Colors_gt_224', 'endobj', 'endstream'] # Giữ nguyên danh sách drop như bạn đề xuất
# Lấy danh sách các cột còn lại để chắc chắn không có lỗi
existing_cols_to_drop = [col for col in features_to_drop if col in df.columns]

In [10]:
print(df.head(5))

   Page  Encrypt  ObjStm  JS  JavaScript  AA  OpenAction  AcroForm  \
0     1        0       0   0           0   0           0         0   
1     1        0       0   0           0   0           0         0   
2     4        0       6   0           0   0           0         0   
3     1        0       0   0           0   0           0         1   
4     6        0      25   0           0   0           0         2   

   JBIG2Decode  RichMedia  ...  stream  endstream  xref  trailer  startxref  \
0            0          0  ...       3          3     2        2          2   
1            0          0  ...       2          2     1        1          1   
2            0          0  ...      41         41     0        0          3   
3            0          0  ...      17         17     2        2          2   
4            0          0  ...     146        146     0        0          4   

                                            filepath          filename  \
0  /home/remnux/Desktop/extrac

In [11]:
# X là các đặc trưng, y là nhãn đã được mã hóa
X = df.drop(columns=existing_cols_to_drop + ['label_numeric']) # Bỏ cả cột label gốc và cột label_numeric khỏi X
y = df['label_numeric'] # Sử dụng cột label_numeric làm biến mục tiêu


In [12]:
X.shape, y.shape

((11101, 20), (11101,))

In [13]:
# Kiểm tra lại các cột của X
print("\nCác đặc trưng được sử dụng cho X:")
print(X.columns.tolist())
print(f"Kích thước của X: {X.shape}")
print(f"Kích thước của y: {y.shape}")
print(f"Phân phối của y (nhãn số): \n{pd.Series(y).value_counts(normalize=True)}")


Các đặc trưng được sử dụng cho X:
['Page', 'Encrypt', 'ObjStm', 'JS', 'JavaScript', 'AA', 'OpenAction', 'AcroForm', 'JBIG2Decode', 'RichMedia', 'Launch', 'EmbeddedFile', 'XFA', 'Colors_gt_224', 'obj', 'stream', 'xref', 'trailer', 'startxref', 'filesize_kb']
Kích thước của X: (11101, 20)
Kích thước của y: (11101,)
Phân phối của y (nhãn số): 
label_numeric
0    0.820377
1    0.179623
Name: proportion, dtype: float64


In [14]:
# Phân chia dữ liệu (Stratified để giữ tỷ lệ lớp)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [15]:
# Chuẩn hóa/Scaling đặc trưng (cho SVM và có thể cho XGBoost/LightGBM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
# Install torch if not already installed
%pip install torch

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")

  Using cached setuptools-80.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached setuptools-80.4.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/865.0 MB ? eta -:--:--Downloading torch-2.7.0-cp313-cp313-manylinux_2_28_x86_64.whl (865.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.0/865.0 MB 5.6 MB/s eta 0:00:0000:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.0/865.0 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 6.1 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 6.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/8.9 MB ? eta -:--:--Downloading nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (8.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
# Lưu scaler để sử dụng sau này (quan trọng cho GAN và đánh giá Metasploit)
joblib.dump(scaler, '/home/nhat/projectcuoiky/models/scaler_baseline.joblib')

['/home/nhat/projectcuoiky/models/scaler_baseline.joblib']

In [21]:
# --- 2. Huấn luyện và Tinh chỉnh Mô hình Scikit-learn ---
cv_stratified = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results_baseline = []

def evaluate_sklearn_model(name, model, X_test_data, y_test_data, is_pytorch_model=False):
    if is_pytorch_model:
        # Xử lý cho mô hình PyTorch (sẽ định nghĩa sau)
        model.eval()
        with torch.no_grad():
            X_test_tensor = torch.FloatTensor(X_test_data).to(device)
            outputs = model(X_test_tensor)
            y_proba = torch.sigmoid(outputs).cpu().numpy().flatten() # Nếu là binary classification với Sigmoid
            y_pred = (y_proba > 0.5).astype(int)
    else:
        y_pred = model.predict(X_test_data)
        y_proba = model.predict_proba(X_test_data)[:, 1]

    print(f"\n--- Kết quả cho mô hình: {name} ---")
    print(classification_report(y_test_data, y_pred, target_names=le.classes_))
    print("Ma trận nhầm lẫn:")
    print(confusion_matrix(y_test_data, y_pred))
    print(f"ROC AUC Score: {roc_auc_score(y_test_data, y_proba):.4f}")
    print(f"Accuracy: {accuracy_score(y_test_data, y_pred):.4f}")
    return {
        'name': name,
        'accuracy': accuracy_score(y_test_data, y_pred),
        'roc_auc': roc_auc_score(y_test_data, y_proba),
        'report': classification_report(y_test_data, y_pred, target_names=le.classes_, output_dict=True)
    }

In [22]:
# a. SVM (Ví dụ với RBF Kernel - bạn có thể thêm Linear)
print("\n--- Đang huấn luyện SVM ---")
param_grid_svm_rbf = {'C': [1, 10], 'kernel': ['rbf'], 'gamma': ['scale', 0.1]}
svm_rbf_grid = GridSearchCV(SVC(probability=True, random_state=42, class_weight='balanced'),
                            param_grid_svm_rbf, cv=cv_stratified, scoring='roc_auc', n_jobs=-1)
svm_rbf_grid.fit(X_train_scaled, y_train)
best_svm_rbf = svm_rbf_grid.best_estimator_
results_baseline.append(evaluate_sklearn_model("SVM (RBF Kernel - Baseline)", best_svm_rbf, X_test_scaled, y_test))
joblib.dump(best_svm_rbf, '/home/nhat/projectcuoiky/models/svm_rbf_baseline.joblib')


--- Đang huấn luyện SVM ---

--- Kết quả cho mô hình: SVM (RBF Kernel - Baseline) ---
              precision    recall  f1-score   support

      benign       0.98      0.99      0.98      2733
   malicious       0.95      0.91      0.93       598

    accuracy                           0.97      3331
   macro avg       0.96      0.95      0.96      3331
weighted avg       0.97      0.97      0.97      3331

Ma trận nhầm lẫn:
[[2703   30]
 [  55  543]]
ROC AUC Score: 0.9845
Accuracy: 0.9745

--- Kết quả cho mô hình: SVM (RBF Kernel - Baseline) ---
              precision    recall  f1-score   support

      benign       0.98      0.99      0.98      2733
   malicious       0.95      0.91      0.93       598

    accuracy                           0.97      3331
   macro avg       0.96      0.95      0.96      3331
weighted avg       0.97      0.97      0.97      3331

Ma trận nhầm lẫn:
[[2703   30]
 [  55  543]]
ROC AUC Score: 0.9845
Accuracy: 0.9745


['/home/nhat/projectcuoiky/models/svm_rbf_baseline.joblib']

In [23]:
# b. Random Forest
print("\n--- Đang huấn luyện Random Forest ---")
param_grid_rf = {'n_estimators': [100, 200], 'max_depth': [10, 20], 'min_samples_leaf': [1, 2], 'class_weight':['balanced']}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf,
                       cv=cv_stratified, scoring='roc_auc', n_jobs=-1)
rf_grid.fit(X_train, y_train) # RF không cần scaled data, nhưng bạn có thể thử cả X_train_scaled
best_rf = rf_grid.best_estimator_
results_baseline.append(evaluate_sklearn_model("Random Forest (Baseline)", best_rf, X_test, y_test))
joblib.dump(best_rf, '/home/nhat/projectcuoiky/models/rf_baseline.joblib')


--- Đang huấn luyện Random Forest ---

--- Kết quả cho mô hình: Random Forest (Baseline) ---
              precision    recall  f1-score   support

      benign       0.99      1.00      0.99      2733
   malicious       0.98      0.96      0.97       598

    accuracy                           0.99      3331
   macro avg       0.99      0.98      0.98      3331
weighted avg       0.99      0.99      0.99      3331

Ma trận nhầm lẫn:
[[2722   11]
 [  26  572]]
ROC AUC Score: 0.9969
Accuracy: 0.9889

--- Kết quả cho mô hình: Random Forest (Baseline) ---
              precision    recall  f1-score   support

      benign       0.99      1.00      0.99      2733
   malicious       0.98      0.96      0.97       598

    accuracy                           0.99      3331
   macro avg       0.99      0.98      0.98      3331
weighted avg       0.99      0.99      0.99      3331

Ma trận nhầm lẫn:
[[2722   11]
 [  26  572]]
ROC AUC Score: 0.9969
Accuracy: 0.9889


['/home/nhat/projectcuoiky/models/rf_baseline.joblib']

In [24]:
# c. XGBOOST
print("\n--- Đang huấn luyện XGBOOST ---")
scale_pos_weight_xgb = np.sum(y_train == 0) / np.sum(y_train == 1)
param_grid_xgb = {'n_estimators': [100, 200], 'max_depth': [5, 7], 'learning_rate': [0.1], 'scale_pos_weight': [scale_pos_weight_xgb]}
xgb_grid = GridSearchCV(xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
                        param_grid_xgb, cv=cv_stratified, scoring='roc_auc', n_jobs=-1)
xgb_grid.fit(X_train, y_train)
best_xgb = xgb_grid.best_estimator_
results_baseline.append(evaluate_sklearn_model("XGBOOST (Baseline)", best_xgb, X_test, y_test))
joblib.dump(best_xgb, '/home/nhat/projectcuoiky/models/xgb_baseline.joblib')


--- Đang huấn luyện XGBOOST ---


/home/nhat/projectcuoiky/.venv/lib/python3.13/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/nhat/projectcuoiky/.venv/lib/python3.13/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant


--- Kết quả cho mô hình: XGBOOST (Baseline) ---
              precision    recall  f1-score   support

      benign       0.99      0.99      0.99      2733
   malicious       0.96      0.97      0.97       598

    accuracy                           0.99      3331
   macro avg       0.98      0.98      0.98      3331
weighted avg       0.99      0.99      0.99      3331

Ma trận nhầm lẫn:
[[2712   21]
 [  20  578]]
ROC AUC Score: 0.9979
Accuracy: 0.9877


/home/nhat/projectcuoiky/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [01:30:43] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


['/home/nhat/projectcuoiky/models/xgb_baseline.joblib']

In [ ]:
# d. LightGBM
print("\n--- Đang huấn luyện LightGBM ---")
scale_pos_weight_lgb = np.sum(y_train == 0) / np.sum(y_train == 1)
param_grid_lgb = {'n_estimators': [100, 200], 'max_depth': [5, 7], 'learning_rate': [0.1], 'num_leaves': [31, 40], 'scale_pos_weight': [scale_pos_weight_lgb]}
lgb_grid = GridSearchCV(lgb.LGBMClassifier(random_state=42),
                        param_grid_lgb, cv=cv_stratified, scoring='roc_auc', n_jobs=-1)
lgb_grid.fit(X_train, y_train)
best_lgb = lgb_grid.best_estimator_
results_baseline.append(evaluate_sklearn_model("LightGBM (Baseline)", best_lgb, X_test, y_test))
joblib.dump(best_lgb, '/home/nhat/projectcuoiky/models/lgb_baseline.joblib')


--- Đang huấn luyện LightGBM ---
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1116, number of negative: 5100
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1116, number of negative: 5100
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099
[LightGBM] [Info] Number of positive: 1117, number of negative: 5099


In [ ]:
# --- 3. Huấn luyện Mô hình MLP bằng PyTorch ---
print("\n--- Đang huấn luyện MLP (PyTorch) ---")

# Chuyển dữ liệu sang PyTorch Tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device) # Cần unsqueeze cho BCEWithLogitsLoss
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1).to(device)

# Tạo DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Định nghĩa kiến trúc MLP
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3) # Thêm Dropout
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3) # Thêm Dropout
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        # Sigmoid sẽ được áp dụng sau hoặc dùng BCEWithLogitsLoss

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu2(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

input_dim = X_train_scaled.shape[1]
hidden_dim1 = 128
hidden_dim2 = 64
output_dim = 1 # Cho binary classification

mlp_model = SimpleMLP(input_dim, hidden_dim1, hidden_dim2, output_dim).to(device)

# Hàm mất mát và Optimizer
# Sử dụng BCEWithLogitsLoss sẽ ổn định hơn và tự xử lý sigmoid
criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# Huấn luyện mô hình
epochs = 50 # Tăng số epochs nếu cần
for epoch in tqdm(range(epochs)):
    mlp_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = mlp_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    # (Tùy chọn) In loss sau mỗi epoch
    if (epoch+1) % 10 == 0:
         print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Đánh giá mô hình MLP (Hàm evaluate_sklearn_model cần được điều chỉnh một chút cho PyTorch)
def evaluate_pytorch_model(name, model, X_test_tensor, y_test_data, le_classes): # Thêm le_classes
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        # Áp dụng sigmoid để lấy xác suất
        y_proba = torch.sigmoid(outputs).cpu().numpy().flatten()
        y_pred = (y_proba > 0.5).astype(int)

    print(f"\n--- Kết quả cho mô hình: {name} ---")
    print(classification_report(y_test_data, y_pred, target_names=le_classes))
    print("Ma trận nhầm lẫn:")
    print(confusion_matrix(y_test_data, y_pred))
    print(f"ROC AUC Score: {roc_auc_score(y_test_data, y_proba):.4f}")
    print(f"Accuracy: {accuracy_score(y_test_data, y_pred):.4f}")
    return {
        'name': name,
        'accuracy': accuracy_score(y_test_data, y_pred),
        'roc_auc': roc_auc_score(y_test_data, y_proba),
        'report': classification_report(y_test_data, y_pred, target_names=le_classes, output_dict=True)
    }

results_baseline.append(evaluate_pytorch_model("MLP (PyTorch - Baseline)", mlp_model, X_test_tensor, y_test, le.classes_))
# Lưu mô hình PyTorch
torch.save(mlp_model.state_dict(), '/home/nhat/projectcuoiky/models/mlp_pytorch_baseline.pth')


# --- 4. So sánh kết quả Baseline ---
print("\n\n--- Tóm tắt kết quả Baseline ---")
baseline_summary_df = pd.DataFrame(results_baseline)
print(baseline_summary_df[['name', 'accuracy', 'roc_auc']])

# In chi tiết F1 và Recall cho lớp 'malicious'
for res in results_baseline:
    print(f"\nChi tiết cho {res['name']}:")
    # Kiểm tra xem 'malicious' có trong report không (dựa vào ánh xạ nhãn)
    malicious_label_str = [k for k, v in label_mapping.items() if v == 1][0] # Tìm tên lớp malicious
    if malicious_label_str in res['report']:
        print(f"  F1-score cho '{malicious_label_str}': {res['report'][malicious_label_str]['f1-score']:.4f}")
        print(f"  Recall cho '{malicious_label_str}': {res['report'][malicious_label_str]['recall']:.4f}")
    else:
        print(f"  Không tìm thấy lớp '{malicious_label_str}' trong báo cáo.")